# **Preprocessing**

In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json
import string
import gdown
from groq import Groq
import os
import re

In [ ]:
file_id = 'FILE_ID'
url = f'https://drive.google.com/uc?export=download&id={file_id}'
gdown.download(url, quiet=False)

In [ ]:
df = pd.read_json("data 2.json")

In [ ]:
part1_df = df[df.model == "api.part1question"]
part2_df = df[df.model == "api.part2question"]
part3_df = df[df.model == "api.part3question"]

part1_answers_df = df[df.model == "school_api.schoolpart1result"]
part2_answers_df = df[df.model == "school_api.schoolpart2result"]
part3_answers_df = df[df.model == "school_api.schoolpart3result"]

parsed_df = df[df.model == "school_api.parsedsession"]

In [ ]:
def splitter(df):
    df_expanded = pd.concat([df.drop(columns=['fields']), df['fields'].apply(pd.Series)], axis=1)
    df_expanded = df_expanded[['pk', 'question_txt']]
    df_expanded = df_expanded.reset_index(drop=True)

    return df_expanded

In [ ]:
def splitter_answers(df):
    df_expanded = pd.concat([df.drop(columns=['fields']), df['fields'].apply(pd.Series)], axis=1)
    df_expanded = df_expanded[["session", 'answer',  'question']]
    df_expanded = df_expanded.dropna(subset=['answer'])
    df_expanded = df_expanded.reset_index(drop=True)

    return df_expanded

In [ ]:
def splitter_parsed(df):
    df_expanded = pd.concat([df.drop(columns=['fields']), df['fields'].apply(pd.Series)], axis=1)
    df_expanded = df_expanded[['session', 'raw_json', "parsed_json", "feedback", "band_score"]]
    df_expanded = df_expanded.reset_index(drop=True)

    return df_expanded

In [ ]:
df_part1_ques_df = splitter(part1_df)
df_part2_ques_df = splitter(part2_df)
df_part3_ques_df = splitter(part3_df)

df_part1_ans_df = splitter_answers(part1_answers_df)
df_part2_ans_df = splitter_answers(part2_answers_df)
df_part3_ans_df = splitter_answers(part3_answers_df)

df_parsed = splitter_parsed(parsed_df)

## **Merging**

In [ ]:
def merged_df(df1, df2):
    merged_df = pd.merge(df1, df2, left_on='pk', right_on='question', how='inner')

    return merged_df

merged_df_1 = merged_df(df_part1_ques_df, df_part1_ans_df)
merged_df_2 = merged_df(df_part2_ques_df, df_part2_ans_df)
merged_df_3 = merged_df(df_part3_ques_df, df_part3_ans_df)

In [ ]:
final_ques_ans = pd.concat([merged_df_1, merged_df_2, merged_df_3], axis=0, ignore_index=True)

## **Saving 100 data in .txt files in data folder**

```
- examiner: <questions by examiner>
- candidate: <answered voice transcript>
------------------ part_1 ---------
- examiner: <questions by examiner>
- candidate: <answered voice transcript>
------------------ part_2 ---------
- examiner: <questions by examiner>
- candidate: <answered voice transcript>
------------------ part_3 ---------
- examiner: <questions by examiner>
```



In [ ]:
output_directory = 'data'
os.makedirs(output_directory, exist_ok=True)

grouped = final_ques_ans.groupby('session')

def split_based_on_pk_order(data):
    parts = []
    part = []

    for i in range(len(data)):
        if i == 0 or data['pk'].iloc[i] == data['pk'].iloc[i-1] + 1:
            part.append(data.iloc[i])
        else:
            parts.append(part)
            part = [data.iloc[i]]

    if part:
        parts.append(part)

    return parts

def format_output(parts):
    output = ""

    for i, part in enumerate(parts):
        output += f"------------------ part_{i+1} ---------\n"
        for row in part:
            questions = row['question_txt']
            answer = row['answer']
            output += f"- examiner: {questions}.\n candidate: {answer}\n"

    return output

for session, data in grouped:

    parts = split_based_on_pk_order(data)
    formatted_output = format_output(parts)

    with open(os.path.join(output_directory, f'session_{session}.txt'), 'w') as file:
        file.write(formatted_output)


print("Processing complete. Files saved in the 'data' directory.")

Processing complete. Files saved in the 'data' directory.


# **Groq (model: llama3-70b-8192)**

In [ ]:
client = Groq(
    api_key="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",
)

## **System Message (Original and Modified version)**

In [ ]:
system_message_original = {
    "role": "system",
    "content": """
    Act like an IELTS Speaking examiner, and you will be given transcripts of IELTS Speaking dialog which you must NEATLY evaluate based on all IELTS criteria. Please, strictly look at the criteria of IELTS Speaking which include: Fluency and coherence, Lexical Resource, Grammar Range, and Accuracy and Pronunciation. For Accuracy and Pronunciation, must consider given Pronunciation Assessment. For each criteria, you should give scores from 0-9 . Provide EXACT overall band score based on all four aspects of IELTS Speaking exam. Besides, you should provide concise feedback STRICTLY following listed rules
Your feedback should be personalized and specific
Mention specific mistakes only
Evaluate Fluency and Suggest how it could be improved
Assess the way a student is using Lexical Resource and quote in which parts it could be boosted with alternatives and collocations.
Carefully examine transition and flow between ideas and suggest better usage of cohesive devices.
PLEASE FOLLOW THE PROVIDED FINAL OUTPUT STRUCTURE
in the output and the EXACT Overall band score pointing to 4 aspects separately as (MUST-HAVE): FLUENCY, VOCABULARY, GRAMMAR, PRONUNCIATION if and only if the conversation contains 3 complete parts. here is the transcript structure:
- examiner: <questions by examiner>
- candidate: <answered voice transcript>
there is the pronunciation_assessment graded by the examiner in natural conversation in the END:
pronunciation_assessment:
- pronunciation_score: <float> out of 100 (final one)
- accuracy_score: <float> out of 100
- fluency_score: <float> out of 100
- completeness_score : <float> out of 100
- mispronounced_words: list of words


Final output structure:
Feedback :<Feedback>

Band score: <>
Fluency: <Fluency>
Vocabulary: <Vocabulary >
Grammar: <Grammar>
Pronunciation: <Pronunciation>

Specific topic-words used by candidate(worth-to-mention):

Suggested less used topic-related words, idioms or collocations :<|eot_id|>
<|start_header_id|>user<|end_header_id|>


- examiner: <questions by examiner>
- candidate: <answered voice transcript>
------------------ part_1 ---------
- examiner: <questions by examiner>
- candidate: <answered voice transcript>
------------------ part_2 ---------
- examiner: <questions by examiner>
- candidate: <answered voice transcript>
------------------ part_3 ---------
- examiner: <questions by examiner>



- candidate: <answered voice transcript><|eot_id|>


<|start_header_id|>assistant<|end_header_id|>
    """
}

In [ ]:
system_message_modified = {
    "role": "system",
    "content": """
    Act as an IELTS Speaking examiner. You will be provided with transcripts of IELTS Speaking dialogues, which you must evaluate NEATLY based on all IELTS criteria. The criteria include: Fluency and Coherence, Lexical Resource, Grammatical Range and Accuracy, and Pronunciation. Consider the provided Pronunciation Assessment for Pronunciation evaluation.

    For each criterion, assign a score from 0 to 9. Calculate the EXACT overall band score based on the four criteria of the IELTS Speaking exam. Additionally, provide concise feedback STRICTLY following the rules outlined below:

    1. Personalize your feedback and make it specific to the candidate's performance.
    2. Mention specific mistakes.
    3. Evaluate Fluency and suggest how it could be improved.
    4. Assess the use of Lexical Resource, quoting specific parts where improvements with alternatives and collocations are possible.
    5. Carefully examine the transition and flow between ideas, suggesting better usage of cohesive devices.

    Ensure you FOLLOW the PROVIDED FINAL OUTPUT STRUCTURE for your evaluation. The overall band score must be separated into the four aspects: FLUENCY, VOCABULARY, GRAMMAR, and PRONUNCIATION. This structure should only be followed if the conversation contains all 3 complete parts. The transcript structure is as follows:

    - examiner: <questions by examiner>
    - candidate: <answered voice transcript>

    If provided, include the pronunciation_assessment at the end:
    pronunciation_assessment:
    - pronunciation_score: <float> out of 100
    - accuracy_score: <float> out of 100
    - fluency_score: <float> out of 100
    - completeness_score : <float> out of 100
    - mispronounced_words: <list of words>

    Final output structure:
    Feedback: <Feedback>

    Band score: <Overall Band Score>
    Fluency: <Fluency Score>
    Vocabulary: <Vocabulary Score>
    Grammar: <Grammar Score>
    Pronunciation: <Pronunciation Score>

    Specific topic-words used by the candidate (worth mentioning):

    Suggested less used topic-related words, idioms, or collocations:

    Transcript Structure:
    ------------------ part_1 ---------
    - examiner: <questions by examiner>
    - candidate: <answered voice transcript>
    ------------------ part_2 ---------
    - examiner: <questions by examiner>
    - candidate: <answered voice transcript>
    ------------------ part_3 ---------
    - examiner: <questions by examiner>
    - candidate: <answered voice transcript>
    """
}


## **One shot example**

In [ ]:
content_user = """
------------------ part_1 ---------
- examiner: How often do you wear jewelry?.
 candidate: well I really like to wear jewelry it just can complete your style and it's really it look very beautiful I can't go out without jewelry and always should be in my always should be I should wear the jewelry especially which my mom gave to me and also the complex complex
- examiner: What type of jewelry do you like to wear?.
 candidate: Well, I really like to wear the chain, bracelet and also the ring from the How to say it? Brilliant, huh? I really interested to wear the jewelry from the beasters
- examiner: Do you usually buy jewelery?.
 candidate: No, I don't like to buy the jewelry because I have the money only to the same jewelry and every time I lose this jewelry which I buy from myself. I never lose the jewelry which someone do present to me.
- examiner: Why do you think some people wear a piece of jewelry for a long time?.
 candidate: I don't know, it may be someone present, a really darling person present to them, this jewelry, or from this jewelry they can, they can, they can remain about something
------------------ part_2 ---------
- examiner: Describe an occasion when you got incorrect information.

 You should say:
 • why you need this information
 • how you got it
 • how you found it was incorrect
 • and how you felt about this experience..
 candidate: Well, I'm going to talk about the occasion when I got incorrect information. Well, it was 2 years ago when I was in the 9th grade and it was the exams after the school graduation exams and after that I should enter the university. Well, we had overall 20 exams but 14 exams was in between 6th of May and my friend said me no that time I haven't I hadn't my own mobile phone and my friend said me that the test exam started at 11. Guys, what do you think and I and this day is came and I went to the school and look there is the no one from the my class there is the another pure another pupils from the another class which they graduated 11th grade but 9th grade which there is no one my friend said me incorrect information at starting time it started at night and when when I came it's just ended I was very angry I don't I didn't know I should cry or smile I mean I should I think I thought that I saw that I should smile because I I wouldn't to I wouldn't to enter I wouldn't I hadn't the exams it was really difficult and what I want why I wanted to cry because I shouldn't to enter the university to you and to entering to university to this room you needed the paper which wrote you have done have done all your exams well what do you mean what do you think I lost the time and just I I shouldn't study at school 10th and 11th grade notice I'm in 11th grade I'm happy that I'm graduating I'm graduating and I believe I strongly believe that this year I will check the time of exams from myself I wouldn't believe to someone
------------------ part_3 ---------
- examiner: What is the difference between providing information by phone and by email?.
 candidate: Well, there is a lot of kinds of professions which related to giving information to others. For example, reporters. The reporters giving information about the news, about another countries, happening in the countries, to the people from the TV. And second is the teachers, teachers giving the knowledge, and they teach the students, and they give information which they know, and they wanted to be their students better than they. And third one is the doctors, especially actually doctors giving the information for the people who come them, who ill, they are write instructions and medicines, in what time, when they should drink their medicines, they write medicines which they can buy.
- examiner: How can people check whether a piece of information is correct or not?.
 candidate: The difference between providing information by phone and by mail is that by mail it's more easier because by mail you can just send text, send information to the people who you want. Not only one people, you can send it in the first time to ten or more people. It's so fast and more comfortable than by phone. These people check their email very fast. By phone it's more difficult than by email. By phone you should call by one person.
- examiner: Do people trust the information online?.
 candidate: Truth. Do people trust the information online? Yes, some people really do trust the information online. They not only trust the information online, they trust it just from the gaps or sentences they listen from other people. But in some cases, there are different people who don't believe the online and the people's sentences. They believe what they saw.
"""

content_assis = """
 "Feedback:\nThe candidate's responses show a basic level of communication with frequent grammatical errors and limited vocabulary. The use of cohesive devices is minimal, and transitions between ideas are often abrupt or unclear.\n\nFluency: The candidate hesitates frequently, repeats words (\"I really like to wear the jewelry\"), and self-corrects often, which disrupts the flow of speech. There are also instances of incomplete sentences and unclear thoughts, especially in Part 2, which affects the overall fluency.\n\nVocabulary: The range of vocabulary is limited, and there are instances of incorrect word choices (\"beasters\" instead of \"beads\" or \"gemstones\"). The candidate could benefit from using a wider range of topic-specific vocabulary and more precise terms.\n\nGrammar: There are numerous grammatical mistakes, including incorrect verb tenses (\"I should wear the jewelry\"), subject-verb agreement issues (\"this jewelry they can, they can, they can remain about something\"), and incorrect sentence structures. The use of modal verbs is also incorrect at times (\"I wouldn't to enter\").\n\nPronunciation: The transcript does not provide enough information to accurately assess pronunciation.\n\nBand score:5.0 \nFluency: 5.0\nVocabulary: 5.0\nGrammar: 4.5\nPronunciation: 5.0\n\nSpecific topic-words used by candidate (worth-to-mention):\n- Jewelry\n- Chain\n- Bracelet\n- Ring\n- Brilliant\n\nSuggested less used topic-related words, idioms or collocations:\n- Accessorize (as a verb, e.g., \"I like to accessorize with jewelry.\")\n- Gemstones (instead of \"brilliant\")\n- Keepsake (for a piece of jewelry with sentimental value)\n- Timepiece (for a watch)\n- Heirloom (for jewelry passed down through generations)\n- \"Lose track of\" (instead of \"lose this jewelry\")\n- \"Misplace\" (a more accurate verb for losing something)\n- \"Cherished gift\" (instead of \"someone present to me\")\n- \"Hold sentimental value\" (instead of \"they can remain about something\")
 """

one_shot_examples = [
    {"role": "user", "content": content_user},
    {"role": "assistant", "content": content_assis}
]

## **Groq Chat Completions**

In [ ]:
directory = '/content/data'

all_feedback = []

In [ ]:
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r') as file:
            transcript = file.read()

            user_message = {
                "role": "user",
                "content": transcript
            }

            chat_completion = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    system_message_modified,
                    *one_shot_examples,
                    user_message
                ],
                max_tokens=1024,
                temperature=0.7
            )

            model_response = chat_completion.choices[0].message.content

            session_number = ''.join(filter(str.isdigit, filename))

            all_feedback.append({
                "session": session_number,
                "feedback": model_response
            })

In [ ]:
with open('all_feedback_responses_with_assit.json', 'w') as output_file:
    json.dump(all_feedback, output_file, indent=4)

print("All feedback responses saved to all_feedback_responses.json.")

# **Final Preprocessing**

In [ ]:
data = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        session_number = ''.join(filter(str.isdigit, filename))
        with open(os.path.join(directory, filename), 'r') as file:
            content = file.read()
            data.append({'session': session_number, 'content': content})

df_txt = pd.DataFrame(data)

In [ ]:
df_json = pd.read_json("/content/all_feedback_responses_with_assit.json")

In [ ]:
df_txt['session'] = df_txt['session'].astype('int64')

In [ ]:
merged_df_full = pd.merge(df_txt, df_json, on='session', how='inner')

In [ ]:
merged_df_full = pd.merge(merged_df_full, df_parsed, on='session', how='left')

In [ ]:
merged_df_full = merged_df_full.rename(columns={'content': 'transcript', 'feedback_x': 'feedback_llama', 'raw_json': 'feedback_gpt', "band_score": "band_score_gpt"})
merged_df_full.drop(columns=['parsed_json', 'feedback_y'], inplace=True)

In [ ]:
def extract_band_score(feedback):
  match = re.search(r"Band score:\s*(\d+\.\d+)", feedback)
  if match:
    return float(match.group(1))
  else:
    return None

merged_df_full['band_score_llama'] = merged_df_full['feedback_llama'].apply(extract_band_score)

In [ ]:
merged_df_full.to_csv('feedback_llama_gpt_with_scores.csv', index=False)

# **Archive (No longer needed)**

In [ ]:
merged_df_full = pd.merge(final_ques_ans, df_parsed, on='session', how='left')

In [ ]:
new_df = merged_df_full.dropna()

In [ ]:
new_df  = new_df.drop(columns=['pk', 'session','question', 'raw_json', 	'parsed_json'])

In [ ]:
count = 0
for index, row in new_df.iterrows():
  if len(row['feedback']) < 10:
    count += 1
print(count)

0


In [ ]:
new_df = new_df[new_df['feedback'].str.len() >= 10]
new_df.reset_index(drop=True, inplace=True)

In [ ]:
shuffled_df = new_df.sample(frac=1)
subset_df = shuffled_df.iloc[:150]
subset_df.to_csv('question_answer_feedback.csv', index=False)